# Machine Learning 2020 Course Projects

## Project Schedule

In this project, you will solve a real-life problem with a dataset. The project will be separated into two phases:

27th May - 9th June: We will give you a training set with target values and a testing set without target. You predict the target of the testing set by trying different machine learning models and submit your best result to us and we will evaluate your results first time at the end of phase 1.

10th June - 24th June: Students stand high in the leader board will briefly explain  their submission in a proseminar. We will also release some general advice to improve the result. You try to improve your prediction and submit final results in the end. We will again ask random group to present and show their implementation.
The project shall be finished by a team of two people. Please find your teammate and REGISTER via [here](https://docs.google.com/forms/d/e/1FAIpQLSf4uAQwBkTbN12E0akQdxfXLgUQLObAVDRjqJHcNAUFwvRTsg/alreadyresponded).

The submission and evaluation is processed by [Kaggle](https://www.kaggle.com/t/426d97d4138b49b2802c2ee0461a18ac).  In order to submit, you need to create an account, please use your team name in the `team tag` on the [kaggle page](https://www.kaggle.com/t/426d97d4138b49b2802c2ee0461a18ac). Two people can submit as a team in Kaggle.

You can submit and test your result on the test set 2 times a day, you will be able to upload your predicted value in a CSV file and your result will be shown on a leaderboard. We collect data for grading at 22:00 on the **last day of each phase**. Please secure your best results before this time.



## Project Description

Car insurance companies are always trying to come up with a fair insurance plan for customers. They would like to offer a lower price to the careful and safe driver while the careless drivers who file claims in the past will pay more. In addition, more safe drivers mean that the company will spend less in operation. However, for new customers, it is difficult for the company to know who the safe driver is. As a result, if a company offers a low price, it bears a high risk of cost. If not, the company loses competitiveness and encourage new customers to choose its competitors.


Your task is to create a machine learning model to mitigate this problem by identifying the safe drivers in new customers based on their profiles. The company then offers them a low price to boost safe customer acquirement and reduce risks of costs. We provide you with a dataset (train_set.csv) regarding the profile (columns starting with ps_*) of customers. You will be asked to predict whether a customer will file a claim (`target`) in the next year with the test_set.csv 

~~You can find the dataset in the `data/final-project-data` folders in the jupyter hub.~~ We also upload dataset to Kaggle and will test your result and offer you a leaderboard in Kaggle. Please find them under the Data tag on the following page:

https://www.kaggle.com/t/426d97d4138b49b2802c2ee0461a18ac

## Phase 1: 26th May - 9th June

### Data Description

In order to take a look at the data, you can use the `describe()` method. As you can see in the result, each row has a unique `id`. `Target` $\in \{0, 1\}$ is whether a user will file a claim in his insurance period. The rest of the 57 columns are features regarding customers' profiles. You might also notice that some of the features have minimum values of `-1`. This indicates that the actual value is missing or inaccessible.


In [171]:
# Quick load dataset and check
import pandas as pd
import os, sys
running_local = True if os.getenv('JUPYTERHUB_USER') is None else False
if not running_local:
    path = "/data/final-project-dataset/"
else:
    path = "./data/"
    !{sys.executable} -m pip install -r requirements.txt

In [172]:
filename = os.path.join(path, "train_set.csv")
data_train = pd.read_csv(filename)
filename = path + "test_set.csv"
data_test = pd.read_csv(filename)

The prefix, e.g. `ind` and `calc`, indicate the feature belongs to similiar groupings. The postfix `bin` indicates binary features and `cat` indicates categorical features. The features without postfix are ordinal or continuous. Similarly, you can check the statistics for testing data:

In [173]:
from tqdm.notebook import tqdm, trange
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from IPython.display import set_matplotlib_formats
from contracts import contract
import sklearn
from sklearn import cluster, datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE


### Handling missing values

In [174]:
# view the missing columns
missing_col = {}
for col in data_train.columns:
    counter = len(data_train[data_train[col] == -1])
    if counter > 0:
        missing_col[col] = counter / len(data_train) * 100
        print('{}\t{:.2f}'.format(col, missing_col[col]))

ps_ind_02_cat	0.03
ps_ind_04_cat	0.01
ps_ind_05_cat	0.98
ps_reg_03	18.13
ps_car_01_cat	0.02
ps_car_02_cat	0.00
ps_car_03_cat	69.07
ps_car_05_cat	44.77
ps_car_07_cat	1.91
ps_car_09_cat	0.09
ps_car_11	0.00
ps_car_12	0.00
ps_car_14	7.15


In [175]:
data_train.drop(columns=[col for col, val in missing_col.items() if val >= 10 and 'cat' not in col], inplace=True)
data_test.drop(columns=[col for col, val in missing_col.items() if val >= 10 and 'cat' not in col], inplace=True)

# transform cat to bin and fill the rest with median values
for col, val in missing_col.items():
    if val >= 10:
        if 'cat' not in col:
            continue
        data_train.loc[data_train[col] != -1, col] = 1
        data_train[col].replace(-1, 0, inplace=True)
        data_train.rename(columns={col: col.replace('cat', 'bin')}, inplace=True)
        print('{}\t-> {}'.format(col, col.replace('cat', 'bin')))
        continue
    median = data_train[data_train[col] != -1][col].median()
    data_train[col].replace(-1, median, inplace=True)

ps_car_03_cat	-> ps_car_03_bin
ps_car_05_cat	-> ps_car_05_bin



### Oversampling with SMOTE

In [176]:
## Select target and features
fea_col = data_train.columns[2:]
data_Y = data_train['target']
data_X = data_train[fea_col]

x_train, x_val, y_train, y_val = train_test_split(data_X, data_Y, test_size = 0.3, shuffle = True)

smote = SMOTE(sampling_strategy='minority')
x_train_smote, y_train_smote = smote.fit_resample(x_train, y_train)
x_train_smote, y_train_smote = np.array(x_train_smote), np.array(y_train_smote)


### Neural Network

In [244]:
def train_neural_network_pytorch(net, inputs, labels, optimizer, criterion, iterations=1000):
    """
    :param net: the neural network object
    :param inputs: numpy array of training data values
    :param labels: numpy array of training data labels 
    :param optimizer: PyTorch optimizer instance
    :param criterion: PyTorch loss function
    :param iterations: number of training steps
    """
    net.train()  # Before training, set the network to training mode

    for iter in trange(iterations):  # loop over the dataset multiple times
        
        # Get the inputs; data is a list of [inputs, labels]
        # Convert to tensors if data is in the form of numpy arrays
        if not torch.is_tensor(inputs):
            inputs = torch.from_numpy(inputs.astype(np.float32)) 
            
        if not torch.is_tensor(labels):
            labels = torch.from_numpy(labels.astype(np.float32))

        # 1. Reset gradients
        optimizer.zero_grad()  
        # 2. Forward
        outputs = net(inputs)
        # 3. Compute the loss
        loss = criterion(outputs.reshape(-1), labels)
        # 4. Backward
        loss.backward()
        # 5. Update parameters
        optimizer.step()
        
    print('Finished Training')

In [245]:
def predict_pytorch(net, X):
    """
    Function for producing network predictions
    """
    
    net.eval()
    
    # Computes probabilities using forward propagation, and classifies to 0/1 using 0.5 as the threshold.
    X = torch.from_numpy(X.astype(np.float32))
    logits = net(X)
    predictions = torch.sigmoid(logits) > 0.5
    
    return predictions

In [246]:
@contract(Y_pred='array[Mx1],M>0',
          Y='array[Mx1],M>0',
          returns='float,>=0.0,<=1.0')
def calc_accuracy(Y_pred, Y):
    """
    Calculates the accuracy of the predictions against the true labels
    (What percent of the predicted labels Y_pred matches the true labels in Y)
    
    param: Y_pred: Predictions of our model (numpy array of shape [m,1] containing 0s and 1s)
    param: Y: Target labels (numpy array of shape [m,output_dim])
    
    returns: accuracy (float between 0.0 and 1.0)  
    """
    
    accuracy = float(np.dot(Y.T,Y_pred) + np.dot((1-Y).T,1-Y_pred))/float(Y.size)
    
    return accuracy

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torchsummary import summary
torch.manual_seed(1234)

class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.output_size = output_size
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Flatten the input x keeping the batch dimension the same
        x = x.reshape(-1, self.input_size)  
        x = F.relu(self.fc1(x))  
        x = F.relu(self.fc2(x))
        x = self.fc3(x)          

        return x  # Return x (logits)

In [257]:
# Define hyperparameters
LEARNING_RATE = 0.001
MOMENTUM = 0.9
MAX_ITERATIONS = 100
INPUT_SIZE = x_train_smote.shape[1]
HIDDEN_SIZE = 37 # tested below, empirical rule ~ mean of the neurons in the input and output layers
OUTPUT_SIZE = 1

In [252]:
# Initialize the network
net = Net(INPUT_SIZE, HIDDEN_SIZE, OUTPUT_SIZE)

# Define the loss criterion and the training algorithm
criterion = nn.BCEWithLogitsLoss()  # Be careful, use binary cross entropy for binary, CrossEntropy for Multi-class
optimizer = optim.SGD(net.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

In [253]:
# Test different hidden sizes
net_list = []

for i in [37,38,39,40]:
    HIDDEN_SIZE = i
    net = Net(INPUT_SIZE, HIDDEN_SIZE, OUTPUT_SIZE)
    criterion = nn.BCEWithLogitsLoss()  # Be careful, use binary cross entropy for binary, CrossEntropy for Multi-class
    optimizer = optim.SGD(net.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
    train_neural_network_pytorch(net, x_train_smote, y_train_smote, optimizer, criterion, MAX_ITERATIONS)
    train_acc = calc_accuracy(predict_pytorch(net, np.array(x_train)).data.numpy(), np.array(y_train).reshape(-1,1))
    test_acc = calc_accuracy(predict_pytorch(net, np.array(x_val)).data.numpy(), np.array(y_val).reshape(-1,1))
    net_list.append((net,test_acc))
    print(f"Train accuracy: {train_acc:.5f}, Test accuracy: {test_acc:.5f}")
    print("---------------------------------------")    


Finished Training
Train accuracy: 0.56516, Test accuracy: 0.56598
---------------------------------------



Finished Training
Train accuracy: 0.63409, Test accuracy: 0.63458
---------------------------------------



Finished Training
Train accuracy: 0.57471, Test accuracy: 0.57654
---------------------------------------



Finished Training
Train accuracy: 0.57776, Test accuracy: 0.57680
---------------------------------------


In [254]:
import operator
# Take network with best accuracy
net = max(net_list, key=operator.itemgetter(1))[0]

train_acc = calc_accuracy(predict_pytorch(net, np.array(x_train)).data.numpy(), np.array(y_train).reshape(-1,1))
test_acc = calc_accuracy(predict_pytorch(net, np.array(x_val)).data.numpy(), np.array(y_val).reshape(-1,1))
print(f"Train accuracy: {train_acc:.5f}, Test accuracy: {test_acc:.5f}")

Train accuracy: 0.63409, Test accuracy: 0.63458


### Cross validation with skorch test

In [286]:
from skorch import NeuralNetClassifier

inputs = x_train_smote
labels = y_train_smote

class ClassifierModule(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ClassifierModule, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.output_size = output_size
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 10)
        self.fc3 = nn.Linear(10, 2)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x, **kwargs):
        x = F.relu(self.fc1(x))  
        x = self.dropout(x)
        x = F.relu(self.fc2(x))  
        x = self.fc3(x)
        return x

class ClassifierModule2(nn.Module):
    def __init__(
            self,
            num_units=38,
            nonlin=F.relu,
            dropout=0.5,
    ):
        super(ClassifierModule2, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin
        self.dropout = dropout

        self.dense0 = nn.Linear(inputs.shape[1], num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(dropout)
        self.dense1 = nn.Linear(num_units, num_units)
        self.output = nn.Linear(num_units, 2)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = F.relu(self.dense1(X))
        X = F.softmax(self.output(X), dim=-1)
        return X

net_kk = NeuralNetClassifier(
    ClassifierModule2,
    max_epochs=10,
    lr=0.01,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
)

if not torch.is_tensor(inputs):
    inputs = torch.from_numpy(inputs.astype(np.float32)) 
            
if not torch.is_tensor(labels):
    labels = torch.from_numpy(labels.astype(np.float32)).long()

net_kk.fit(inputs, labels)

  epoch    train_loss    valid_acc    valid_loss      dur
-------  ------------  -----------  ------------  -------
      1        0.6478       0.6489        0.6184  15.5456
      2        0.5796       0.7519        0.5302  15.7525
      3        0.5294       0.7641        0.5226  16.1430
      4        0.4932       0.7282        0.5487  17.3121
      5        0.4649       0.7218        0.5240  15.0429
      6        0.4448       0.7349        0.5059  14.3823
      7        0.4307       0.7903        0.4856  13.9336
      8        0.4146       0.7052        0.5361  15.5815
      9        0.4009       0.7624        0.4922  14.1226
     10        0.3921       0.7393        0.5125  14.1161


<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule2(
    (dense0): Linear(in_features=56, out_features=38, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (dense1): Linear(in_features=38, out_features=38, bias=True)
    (output): Linear(in_features=38, out_features=2, bias=True)
  ),
)

In [292]:
pred_train = net_kk.predict_proba(torch.from_numpy(np.array(x_val.astype(np.float32))))
train_acc = calc_accuracy(np.array(list(map(lambda x: 0 if x[0]>0.5 else 1, pred_train))).reshape(-1,1), np.array(y_val).reshape(-1,1))
print(f"Train accuracy: {train_acc:.5f}, Test accuracy: {test_acc:.5f}")

Train accuracy: 0.59654, Test accuracy: 0.63458


In [281]:
print(np.array(list(map(lambda x: 0 if x[0]>0.5 else 1, pred_train))))

[0 0 0 ... 1 1 1]


### Submission

Please only submit the csv files with predicted outcome with its id and target [here](https://www.kaggle.com/t/b3dc81e90d32436d93d2b509c98d0d71). Your column should only contain `0` and `1`.

In [135]:
data_test_X = data_test.drop(columns=['id'])
y_target = np.array(predict_pytorch(net, np.array(data_test_X))).astype(int)

In [136]:
data_out = pd.DataFrame(data_test['id'].copy())
data_out.insert(1, "target", y_target, True) 
data_out.to_csv('./data/submission.csv',index=False)

In [137]:
data_out

,id,target
0,100000,0
1,100001,0
2,100002,1
3,100003,0
4,100004,0
...,...,...
148795,248795,1
148796,248796,0
148797,248797,0
148798,248798,0
